In [1]:
import wrds
import pandas as pd
import numpy as np
import datetime as dt
from fuzzywuzzy import fuzz
db = wrds.Connection(wrds_username='jhirschi')

Loading library list...
Done


### Initial Exploration of the Datasets available to us from WRDS

In [52]:
db.list_libraries()

['wrds_lib_internal',
 'ppubsamp_d4d_new',
 'ktmine',
 'levin',
 'lspdsamp',
 'ciqsamp_pplintel_new',
 'ciqsamp_common_new',
 'contrib_char_returns_new',
 'taq',
 'secsamp_all_new',
 'compd',
 'compgd',
 'preqsamp_all_new',
 'compbd',
 'compsegd',
 'crspa',
 'reprisk',
 'rpnasamp',
 'secsamp',
 'dmef',
 'taqmsamp',
 'snlsamp',
 'sustsamp',
 'toyo',
 'taqm_2017',
 'trsdcgs',
 'twoiq',
 'wcai',
 'wqa',
 'wrdsapps',
 'wrdsrpts',
 'wappsamp',
 'taqmsec',
 'taqm_2019',
 'trws',
 'tr_13f',
 'bank',
 'blab',
 'compmcur',
 'ftsesamp',
 'gmi',
 'hbase',
 'ifgr',
 'ims',
 'cisdm',
 'issm',
 'nastraq',
 'pacap',
 'ppublica',
 'public',
 'rent',
 'snl',
 'sprat',
 'sustain',
 'centris',
 'emdb',
 'eureka',
 'evts',
 'govpx',
 'gsi',
 'govpxsmp',
 'hbsamp',
 'block_all_new',
 'fisd',
 'etfgsamp',
 'infogroupsamp_residential',
 'ktmine_patents_samp',
 'factset',
 'wrdssec',
 'infogroupsamp_business',
 'compm',
 'ims_obp_trial',
 'ifgrsamp',
 'imssamp',
 'compseg',
 'ravenpack_trial',
 'iri',
 'ktsam

In [4]:
db.list_tables(library='ibes')

['igaact',
 'igachg',
 'igactryactdata',
 'igactryestdata',
 'igaebgdates',
 'igaest',
 'igaindactdata',
 'igaindestdata',
 'igainfo',
 'igarussactdata',
 'igarussestdata',
 'igaspactdata',
 'igaspestdata',
 'igatopixactdata',
 'igatopixestdata',
 'act_epsint',
 'act_epsus',
 'actg',
 'actgu',
 'actpsum_epsint',
 'actpsum_epsus',
 'actpsumu_epsint',
 'actpsumu_epsus',
 'actpsumu_xepsint',
 'actpsumu_xepsus',
 'actpsum_xepsint',
 'actpsum_xepsus',
 'actu_epsint',
 'actu_epsus',
 'actu_xepsint',
 'actu_xepsus',
 'act_xepsint',
 'act_xepsus',
 'adj',
 'adjsum',
 'cfnsum',
 'curr',
 'currnew',
 'det_epsint',
 'det_epsus',
 'detsecd_sepint',
 'detsecd_xsepint',
 'detu_epsint',
 'detu_epsus',
 'detusecd_sepint',
 'detusecd_xsepint',
 'detu_xepsint',
 'detu_xepsus',
 'det_xepsint',
 'det_xepsus',
 'eurx',
 'exc_epsint',
 'exc_epsus',
 'excsecd_sepint',
 'excsecd_xsepint',
 'excu_epsint',
 'excu_epsus',
 'excusecd_sepint',
 'excusecd_xsepint',
 'excu_xepsint',
 'excu_xepsus',
 'exc_xepsint',
 

In [5]:
#a quick way to find out which schemas we actually have access to in WRDS Compustat
libs = db.list_tables(library='ibes')
for lib in libs:
    try:
        db.get_table(library='ibes', table=lib, obs=10)
        print('Success!! This table works: ', lib)
    except Exception:
        pass

Success!! This table works:  act_epsint
Success!! This table works:  act_epsus
Success!! This table works:  actg
Success!! This table works:  actgu
Success!! This table works:  actpsum_epsint
Success!! This table works:  actpsum_epsus
Success!! This table works:  actpsumu_epsint
Success!! This table works:  actpsumu_epsus
Success!! This table works:  actpsumu_xepsint
Success!! This table works:  actpsumu_xepsus
Success!! This table works:  actpsum_xepsint
Success!! This table works:  actpsum_xepsus
Success!! This table works:  actu_epsint
Success!! This table works:  actu_epsus
Success!! This table works:  actu_xepsint
Success!! This table works:  actu_xepsus
Success!! This table works:  act_xepsint
Success!! This table works:  act_xepsus
Success!! This table works:  adj
Success!! This table works:  adjsum
Success!! This table works:  cfnsum
Success!! This table works:  curr
Success!! This table works:  currnew
Success!! This table works:  det_epsint
Success!! This table works:  det_ep

In [2]:
db.list_tables(library='comp')

['filings',
 'bank_aacctchg',
 'bank_adesind',
 'bank_afnd1',
 'bank_afnd2',
 'bank_afnddc1',
 'bank_afnddc2',
 'bank_afntind',
 'bank_funda',
 'bank_funda_fncd',
 'bank_fundq',
 'bank_fundq_fncd',
 'bank_iacctchg',
 'bank_idesind',
 'bank_ifndq',
 'bank_ifndytd',
 'bank_ifntq',
 'bank_ifntytd',
 'bank_names',
 'bank_namesq',
 'g_chars',
 'g_co_aaudit',
 'g_co_adesind',
 'g_co_afnd1',
 'g_co_afnd2',
 'g_co_afnddc1',
 'g_co_afnddc2',
 'g_co_afntind1',
 'g_co_afntind2',
 'g_co_ainvval',
 'g_co_hgic',
 'g_co_iaudit',
 'g_co_idesind',
 'g_co_ifndq',
 'g_co_ifndsa',
 'g_co_ifndytd',
 'g_co_ifntq',
 'g_co_ifntsa',
 'g_co_ifntytd',
 'g_co_industry',
 'g_co_ipcd',
 'g_company',
 'g_co_offtitl',
 'g_currency',
 'g_ecind_desc',
 'g_ecind_mth',
 'g_exrt_dly',
 'g_exrt_mth',
 'g_funda',
 'g_funda_fncd',
 'g_fundq',
 'g_fundq_fncd',
 'g_idxcst_his',
 'g_idx_daily',
 'g_idx_index',
 'g_idx_mth',
 'g_names',
 'g_names_ix',
 'g_names_ix_cst',
 'g_namesq',
 'g_sec_adesind',
 'g_sec_adjfact',
 'g_sec_af

In [3]:
#a quick way to find out which schemas we actually have access to in WRDS Compustat
libs = db.list_tables(library='comp')
for lib in libs:
    try:
        db.get_table(library='comp', table=lib, obs=10)
        print('Success!! This table works: ', lib)
    except Exception:
        pass


Success!! This table works:  bank_aacctchg
Success!! This table works:  bank_adesind
Success!! This table works:  bank_afnd1
Success!! This table works:  bank_afnd2
Success!! This table works:  bank_afnddc1
Success!! This table works:  bank_afnddc2
Success!! This table works:  bank_afntind
Success!! This table works:  bank_funda
Success!! This table works:  bank_funda_fncd
Success!! This table works:  bank_fundq
Success!! This table works:  bank_fundq_fncd
Success!! This table works:  bank_iacctchg
Success!! This table works:  bank_idesind
Success!! This table works:  bank_ifndq
Success!! This table works:  bank_ifndytd
Success!! This table works:  bank_ifntq
Success!! This table works:  bank_ifntytd
Success!! This table works:  bank_names
Success!! This table works:  bank_namesq
Success!! This table works:  g_chars
Success!! This table works:  g_co_aaudit
Success!! This table works:  g_co_adesind
Success!! This table works:  g_co_afnd1
Success!! This table works:  g_co_afnd2
Success!!

Success!! This table works:  r_inactvcd
Success!! This table works:  r_incstats
Success!! This table works:  r_indfmt
Success!! This table works:  r_indsec
Success!! This table works:  r_invval
Success!! This table works:  r_issuetyp
Success!! This table works:  r_majidxcl
Success!! This table works:  r_mic_codes
Success!! This table works:  r_naiccd
Success!! This table works:  r_notetype
Success!! This table works:  r_ntsubtype
Success!! This table works:  r_offcrso
Success!! This table works:  r_ogmethod
Success!! This table works:  r_opinions
Success!! This table works:  r_prc_stat
Success!! This table works:  r_qsrcdoc
Success!! This table works:  r_secannfn
Success!! This table works:  r_sec_stat
Success!! This table works:  r_sectors
Success!! This table works:  r_siccd
Success!! This table works:  r_spiicd
Success!! This table works:  r_spmicd
Success!! This table works:  r_statalrt
Success!! This table works:  r_states
Success!! This table works:  r_stko
Success!! This table w

In [20]:
#a quick way to find out which schemas we actually have access to in WRDS CRSP
libs = db.list_tables(library='crsp')
for lib in libs:
    try:
        db.get_table(library='crsp', table=lib, obs=10)
        print('Success!! This table works: ', lib)
    except Exception:
        pass


Success!! This table works:  contact_info
Success!! This table works:  crsp_cik_map
Success!! This table works:  crsp_portno_map
Success!! This table works:  daily_nav
Success!! This table works:  daily_nav_ret
Success!! This table works:  daily_returns
Success!! This table works:  dividends
Success!! This table works:  front_load
Success!! This table works:  front_load_det
Success!! This table works:  front_load_grp
Success!! This table works:  fund_fees
Success!! This table works:  fund_flows
Success!! This table works:  fund_hdr
Success!! This table works:  fund_hdr_hist
Success!! This table works:  fund_names
Success!! This table works:  fund_style
Success!! This table works:  fund_summary
Success!! This table works:  fund_summary2
Success!! This table works:  holdings
Success!! This table works:  mfdbname
Success!! This table works:  monthly_nav
Success!! This table works:  monthly_returns
Success!! This table works:  monthly_tna
Success!! This table works:  monthly_tna_ret_nav
Su

### Begin filtering list of unique tickers/cusips/identifiers

In [28]:
db.get_table(library='crsp', table='stocknames', obs=10)

,permno,permco,namedt,nameenddt,cusip,ncusip,ticker,comnam,hexcd,exchcd,siccd,shrcd,shrcls,st_date,end_date,namedum
0,10000.0,7952.0,1986-01-07,1987-06-11,68391610,68391610,OMFGA,OPTIMUM MANUFACTURING INC,3.0,3.0,3990.0,10.0,A,1986-01-31,1987-06-30,2.0
1,10001.0,7953.0,1986-01-09,1993-11-21,36720410,39040610,GFGC,GREAT FALLS GAS CO,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
2,10001.0,7953.0,1993-11-22,2008-02-04,36720410,29274A10,EWST,ENERGY WEST INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
3,10001.0,7953.0,2008-02-05,2009-08-03,36720410,29274A20,EWST,ENERGY WEST INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
4,10001.0,7953.0,2009-08-04,2009-12-17,36720410,29269V10,EGAS,ENERGY INC,2.0,3.0,4920.0,11.0,None,1986-01-31,2017-08-31,2.0
5,10001.0,7953.0,2009-12-18,2010-07-08,36720410,29269V10,EGAS,ENERGY INC,2.0,2.0,4925.0,11.0,None,1986-01-31,2017-08-31,2.0
6,10001.0,7953.0,2010-07-09,2017-08-03,36720410,36720410,EGAS,GAS NATURAL INC,2.0,2.0,4925.0,11.0,None,1986-01-31,2017-08-31,2.0
7,10002.0,7954.0,1986-01-10,1993-09-29,05978R10,60740110,MBNC,MOBILE NATIONAL CORP,3.0,3.0,6710.0,10.0,None,1986-01-31,2013-02-28,2.0
8,10002.0,7954.0,1993-09-30,2002-05-14,05978R10,83623410,SABC,SOUTH ALABAMA BANCORPORATION IN,3.0,3.0,6710.0,11.0,None,1986-01-31,2013-02-28,2.0
9,10002.0,7954.0,2002-05-15,2013-02-15,05978R10,05978R10,BTFG,BANCTRUST FINANCIAL GROUP INC,3.0,3.0,6020.0,11.0,None,1986-01-31,2013-02-28,2.0


In [78]:
#make initial pull to get identifying info for the companies in the db that currently exist
#then we group them by the exchange they are on
stk_names = db.raw_sql(''' select permno, permco, cusip, ncusip, ticker, comnam, exchcd, namedt, nameenddt, st_date, end_date from crsp.stocknames where end_date = '2019-12-31' ''')
stk_names[['ticker', 'exchcd']].groupby('exchcd').count()

,ticker
exchcd,
-2.0,8
-1.0,40
0.0,0
1.0,4455
2.0,1458
3.0,5806
4.0,2233
5.0,363
6.0,1


Will only look at companies on exchanges 1 - 5

In [79]:
#add filters to get our selection of companies for which we will need matches
#each row is the most recent version of the company name/ticker/cusip of all the available versions
#this is our "names" dataframe
names = stk_names.loc[(~stk_names['ticker'].isnull())&(stk_names['exchcd'].isin([1,2,3,4,5]))].copy()
names.drop_duplicates(subset=['permno', 'permco', 'cusip', 'ncusip', 'ticker'], keep='first', inplace=True)
names.drop_duplicates(subset=['permno', 'permco', 'cusip'], keep='last', inplace=True)
names

,permno,permco,cusip,ncusip,ticker,comnam,exchcd,namedt,nameenddt,st_date,end_date
0,10026.0,7976.0,46603210,46603210,JJSF,J & J SNACK FOODS CORP,3.0,1986-02-04,2019-12-31,1986-02-28,2019-12-31
12,10028.0,7978.0,29402E10,29402E10,ELA,ENVELA CORP,2.0,2019-12-18,2019-12-31,1986-02-28,2019-12-31
13,10032.0,7980.0,72913210,72913210,PLXS,PLEXUS CORP,3.0,1986-02-05,2019-12-31,1986-02-28,2019-12-31
16,10044.0,7992.0,77467X10,77467X10,RMCF,ROCKY MOUNTAIN CHOC FAC INC NEW,3.0,2015-03-02,2019-12-31,1986-02-28,2019-12-31
21,10051.0,7999.0,41043F20,41043F20,HGR,HANGER ORTHOPEDIC GROUP INC,2.0,1991-05-01,1998-12-14,1986-02-28,2019-12-31
...,...,...,...,...,...,...,...,...,...,...,...
14995,93426.0,53443.0,92835K10,92835K10,VPG,VISHAY PRECISION GROUP INC,1.0,2010-07-07,2019-12-31,2010-07-30,2019-12-31
14996,93427.0,53445.0,G3323L10,G3323L10,FN,FABRINET,1.0,2010-06-25,2019-12-31,2010-06-30,2019-12-31
14997,93429.0,53447.0,12503M10,12503M10,CBOE,C B O E HOLDINGS INC,3.0,2010-06-15,2017-10-17,2010-06-30,2019-12-31
15000,93434.0,53427.0,78513510,78513510,SANW,S & W SEED CO,3.0,2010-06-14,2019-12-31,2010-06-30,2019-12-31


Needed to filter to unique permno/permco/cusip. These identifiers will enable us to match data across the databases, despite the fact that companies relist themselves on different exchanges and change tickers regularly

### Below, will match each company with it's fiscal dates to figure out when its quarterly financial statements are released

##### Get the GVKEY matches, evaluate quality of matches and merge into single table

In [16]:
def get_gvkey_match(cusip):
    df= db.raw_sql('''
    select a.conm, a.gvkey, a.naics, b.cusip as long_cusip, left(b.cusip, 8) as cusip, b.tic from comp.company a 
    left join comp.security b
    on a.gvkey = b.gvkey
    where left(b.cusip, 8) ='{}' '''.format(cusip))
    return df

In [67]:
def evaluate_match(comp, crsp):
    #evaluate precision of cusip match (only one row), ticker match, and company name match
    if len(comp) == 1 and comp['cusip'].iloc[0]==crsp.cusip:
        comp['cusip_score']=1
        comp['ticker_match_score']=fuzz.ratio(comp['tic'].iloc[0], crsp.ticker)
        comp['name_match_score']=fuzz.partial_ratio(comp['conm'].iloc[0], crsp.comnam)
    else:
        comp['cusip_score']=0
    return comp

In [61]:
def get_matches_from_names(names): #the loop function. Requires a "names" dataframe
    match = []
    for ent in names.itertuples():
        temp = get_gvkey_match(ent.cusip)
        temp = evaluate_match(temp, ent)
        match.append(temp)
    match = pd.concat(match)
    names = names.merge(match, how = 'left', on='cusip')
    return names

In [75]:
def clean_up_matches(names):
    names['average_match_score']=names[['ticker_match_score', 'name_match_score']].mean(axis=1)
    names = names.loc[(names['cusip_score']==1) & (names['average_match_score']>75)].copy()
    names = names[['permno', 'permco', 'ncusip', 'cusip', 'ticker', 'comnam', 'tic', 'conm', 'gvkey', 'long_cusip', 'naics', 'exchcd', 'namedt', 'nameenddt', 'st_date', 'end_date', 'cusip_score', 'ticker_match_score', 'name_match_score', 'average_match_score']]
    return names

In [80]:
#get all the matches for the CRSP companies in the "names" dataframe
names = get_matches_from_names(names)
names = clean_up_matches(names)

In [82]:
names[['ticker_match_score', 'name_match_score', 'average_match_score']].mean(axis=0)

ticker_match_score     99.701990
name_match_score       88.224279
average_match_score    93.963134
dtype: float64

In [83]:
names.to_csv('company_names_matched.csv', index=0)

In [86]:
names.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6483 entries, 0 to 7878
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   permno               6483 non-null   float64
 1   permco               6483 non-null   float64
 2   ncusip               6483 non-null   object 
 3   cusip                6483 non-null   object 
 4   ticker               6483 non-null   object 
 5   comnam               6483 non-null   object 
 6   tic                  6483 non-null   object 
 7   conm                 6483 non-null   object 
 8   gvkey                6483 non-null   object 
 9   long_cusip           6483 non-null   object 
 10  naics                6119 non-null   object 
 11  exchcd               6483 non-null   float64
 12  namedt               6483 non-null   object 
 13  nameenddt            6483 non-null   object 
 14  st_date              6483 non-null   object 
 15  end_date             6483 non-null   o

##### Get the quarterly statement dates

In [20]:
db.describe_table(library='comp', table='fundq').name.tolist()

Approximately 1817440 rows in comp.fundq.


['gvkey',
 'datadate',
 'fyearq',
 'fqtr',
 'fyr',
 'indfmt',
 'consol',
 'popsrc',
 'datafmt',
 'tic',
 'cusip',
 'conm',
 'acctchgq',
 'acctstdq',
 'adrrq',
 'ajexq',
 'ajpq',
 'bsprq',
 'compstq',
 'curcdq',
 'curncdq',
 'currtrq',
 'curuscnq',
 'datacqtr',
 'datafqtr',
 'finalq',
 'ogmq',
 'rp',
 'scfq',
 'srcq',
 'staltq',
 'updq',
 'apdedateq',
 'fdateq',
 'pdateq',
 'rdq',
 'acchgq',
 'acomincq',
 'acoq',
 'actq',
 'altoq',
 'ancq',
 'anoq',
 'aociderglq',
 'aociotherq',
 'aocipenq',
 'aocisecglq',
 'aol2q',
 'aoq',
 'apq',
 'aqaq',
 'aqdq',
 'aqepsq',
 'aqpl1q',
 'aqpq',
 'arcedq',
 'arceepsq',
 'arceq',
 'atq',
 'aul3q',
 'billexceq',
 'capr1q',
 'capr2q',
 'capr3q',
 'capsftq',
 'capsq',
 'ceiexbillq',
 'ceqq',
 'cheq',
 'chq',
 'cibegniq',
 'cicurrq',
 'ciderglq',
 'cimiiq',
 'ciotherq',
 'cipenq',
 'ciq',
 'cisecglq',
 'citotalq',
 'cogsq',
 'csh12q',
 'cshfd12',
 'cshfdq',
 'cshiq',
 'cshopq',
 'cshoq',
 'cshprq',
 'cstkcvq',
 'cstkeq',
 'cstkq',
 'dcomq',
 'dd1q',
 'derac

In [3]:
def get_5_years_of_earnings(gvkey, ticker, long_cusip):
    df = db.raw_sql('''
    select gvkey, tic, cusip as long_cusip, fyr, fqtr, fyearq, datadate, rdq 
    from comp.fundq 
    where tic = '{0}'
    and gvkey= '{1}'
    and cusip = '{2}' 
    and datadate > '2015-06-01'
    order by rdq asc
    '''.format(ticker, gvkey, long_cusip))
    return df

In [100]:
#example of how earnings events appear
get_5_years_of_earnings('012825', 'JJSF', '466032109')

,gvkey,tic,cusip,fyr,fqtr,fyearq,datadate,rdq
0,012825,JJSF,466032109,9.0,3.0,2015.0,2015-06-30,2015-07-27
1,012825,JJSF,466032109,9.0,4.0,2015.0,2015-09-30,2015-11-05
2,012825,JJSF,466032109,9.0,1.0,2016.0,2015-12-31,2016-01-25
3,012825,JJSF,466032109,9.0,2.0,2016.0,2016-03-31,2016-04-25
4,012825,JJSF,466032109,9.0,3.0,2016.0,2016-06-30,2016-07-25
5,012825,JJSF,466032109,9.0,4.0,2016.0,2016-09-30,2016-11-03
6,012825,JJSF,466032109,9.0,1.0,2017.0,2016-12-31,2017-01-23
7,012825,JJSF,466032109,9.0,2.0,2017.0,2017-03-31,2017-04-24
8,012825,JJSF,466032109,9.0,3.0,2017.0,2017-06-30,2017-07-24
9,012825,JJSF,466032109,9.0,4.0,2017.0,2017-09-30,2017-11-09


In [102]:
def build_statements_table(names):
    earnings = []
    temp = names[['gvkey', 'long_cusip', 'tic']].values
    for ent in temp:
        dates = get_5_years_of_earnings(ent[0], ent[2], ent[1])
        earnings.append(dates)
    earnings = pd.concat(earnings)
    return earnings        

In [103]:
earnings = build_statements_table(names)

In [121]:
#consolidate the final earnings table to all needed points for price pulls
final_earnings = names.merge(earnings, how='left', on=['gvkey', 'tic', 'long_cusip'])
final_earnings=final_earnings.loc[(~final_earnings['rdq'].isnull())&(final_earnings['rdq']<='2019-12-20')].copy()
final_earnings.drop(['ncusip', 'tic', 'conm', 'long_cusip', 'namedt', 'nameenddt', 'st_date', 'end_date',
                     'cusip_score', 'ticker_match_score', 'name_match_score', 'average_match_score'], axis=1, inplace=True)
#final_earnings.to_csv('earnings_dates.csv', index=False)
final_earnings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79608 entries, 0 to 118403
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   permno    79608 non-null  float64
 1   permco    79608 non-null  float64
 2   cusip     79608 non-null  object 
 3   ticker    79608 non-null  object 
 4   comnam    79608 non-null  object 
 5   gvkey     79608 non-null  object 
 6   naics     79608 non-null  object 
 7   exchcd    79608 non-null  float64
 8   fyr       79608 non-null  float64
 9   fqtr      79557 non-null  float64
 10  fyearq    79608 non-null  float64
 11  datadate  79608 non-null  object 
 12  rdq       79608 non-null  object 
dtypes: float64(6), object(7)
memory usage: 8.5+ MB


In [2]:
names = pd.read_csv('company_names_matched.csv')
final_earnings=pd.read_csv('earnings_dates.csv')

### Below will perform analysis on all companies in our list so far to determine what a threshold of a "big move" around earnings time is. We will begin building a labeled dataset as we engineer features to input into our final model

##### Determining threshold for what moves at earnings time are of interest

We don't want to look at moves that are too big (could be resulting from a one time corporate action, a price split/reverse split, etc). We also don't want to look at moves that are big % wise but are too small nominally to be of much use in an options play. Looking at a sweet spot where there is atleast a certain size % change and also a big enough $ change, but not so big as to be a result of something other than what is a pure result of the earnings results

In [48]:
db.describe_table(library='crsp', table='dsf')

Approximately 96285900 rows in crsp.dsf.


,name,nullable,type
0,cusip,True,VARCHAR
1,permno,True,DOUBLE PRECISION
2,permco,True,DOUBLE PRECISION
3,issuno,True,DOUBLE PRECISION
4,hexcd,True,DOUBLE PRECISION
5,hsiccd,True,DOUBLE PRECISION
6,date,True,DATE
7,bidlo,True,DOUBLE PRECISION
8,askhi,True,DOUBLE PRECISION
9,prc,True,DOUBLE PRECISION


In [5]:
def get_20_trading_days(date,permno, cusip):
    date = dt.datetime.strptime(date, '%Y-%m-%d').date()
    st_dt = date+dt.timedelta(days=-14)
    end_dt = date+dt.timedelta(days=14)
    df = db.raw_sql('''
    select cusip, permno, date, prc, vol/1000 as vol, shrout, cfacpr, cfacshr, numtrd, 
    (prc * cfacpr) as adj_prc, (shrout *cfacshr) as adj_shares, (vol/(shrout*cfacshr*1000)) as pct_traded,
    case when prc != (prc*cfacpr) then 1 else 0 end as adj
    from crsp.dsf
    where date >= '{0}'
    and date <= '{1}'
    and permno = {2}
    and cusip = '{3}'
    '''.format(st_dt, end_dt, permno, cusip))
    return df
    

In [ ]:
#taking a random sample of 1000 earnings events to test different nominal and percentage changes for my thresholds
#test for daily returns, 2 day returns, 5 day returns, 10 day returns, and 20 day returns. 
#As well as volatility over the entire period

#graph some histograms of the distributions of your samples for volatility, and for returns, and nominal changes. 
#

In [7]:
trad = get_20_trading_days('2019-10-30' , 14593, '03783310')

In [80]:
def evaluate_earnings(date, permno, cusip, pct_thresh=0.06, nom_thresh=6, cum_pct_thresh_1=0.1, 
                      cum_nom_thresh_1=10, cum_pct_thresh_2=0.1, cum_nom_thresh_2=10):
    df = get_20_trading_days(date, permno, cusip)
    if df.empty:
        return ['Error']
    
    days=len(df['adj_prc'])
    if days<=7:
        return ['Error']
    
    #evaluates the pct and nom diffs
    try:
        pct_diff = df['adj_prc'].pct_change()
        nom_diff = df['adj_prc'].diff()
        max_pct_diff = pct_diff.max()
        max_nom_diff=nom_diff.max()
    except Exception:
        return ['Error']
    
    #evaluates the cumulate pct and nom diffs
    start, end = 0, 4
    cum_pct_diff = []
    cum_nom_diff = []
    while end< days:
        cum_pct_diff.append(pct_diff.iloc[start:end].sum())
        cum_nom_diff.append(nom_diff.iloc[start:end].sum())
        start+=1
        end+=1
    max_cum_pct_diff = max(cum_pct_diff)
    max_cum_nom_diff = max(cum_nom_diff)
    
    #evaluates the cumulate pct and nom diffs level 2
    start, end = 0, 7
    cum_pct_diff_2 = []
    cum_nom_diff_2 = []
    while end< days:
        cum_pct_diff_2.append(pct_diff.iloc[start:end].sum())
        cum_nom_diff_2.append(nom_diff.iloc[start:end].sum())
        start+=1
        end+=1
    max_cum_pct_diff_2 = max(cum_pct_diff_2)
    max_cum_nom_diff_2 = max(cum_nom_diff_2)
    
    #boolean checks
    move_pct = max_pct_diff > pct_thresh
    move_cum_pct_1 = max_cum_pct_diff > cum_pct_thresh_1
    move_cum_pct_2 = max_cum_pct_diff_2 > cum_pct_thresh_2
    
    move_nom = max_nom_diff > nom_thresh
    move_cum_nom_1 = max_cum_nom_diff > cum_nom_thresh_1
    move_cum_nom_2 = max_cum_nom_diff_2 > cum_nom_thresh_2
    
    return move_pct, move_nom, move_cum_pct_1, move_cum_nom_1, move_cum_pct_2, move_cum_nom_2, 

In [50]:
test = get_20_trading_days('2018-04-23', 56641.0, '02927U20' )

In [51]:
test.empty

True

In [77]:
sample = final_earnings.sample(n=600)
sample

,permno,permco,cusip,ticker,comnam,gvkey,naics,exchcd,fyr,fqtr,fyearq,datadate,rdq
57923,85337.0,32078.0,45646310,IBA,INDUSTRIAS BACHOCO S A DE C V,221545,311615,1.0,12.0,3.0,2015.0,2015-09-30,2015-10-22
44471,76076.0,10486.0,17275R10,CSCO,CISCO SYSTEMS INC,20779,334210,3.0,7.0,1.0,2019.0,2018-10-31,2018-11-14
12700,14521.0,54772.0,89628810,TNET,TRINET GROUP INC,140988,518210,1.0,12.0,2.0,2018.0,2018-06-30,2018-07-30
69655,90427.0,45732.0,88268110,TXRH,TEXAS ROADHOUSE INC,160376,722511,3.0,12.0,4.0,2017.0,2017-12-31,2018-02-20
31169,18426.0,56666.0,Y2067610,DSSI,DIAMOND S SHIPPING INC,34956,483111,1.0,12.0,4.0,2018.0,2018-12-31,2019-03-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74295,91742.0,51660.0,83558L20,SNOA,SONOMA PHARMACEUTICALS INC,175424,325412,3.0,3.0,1.0,2015.0,2015-06-30,2015-07-30
43775,75416.0,21995.0,74971D10,RPT,R P T REALTY,15111,531120,1.0,12.0,2.0,2015.0,2015-06-30,2015-07-21
27503,17251.0,56220.0,65441V10,NINE,NINE ENERGY SERVICES INC,31507,213112,1.0,12.0,3.0,2018.0,2018-09-30,2018-11-12
51253,80498.0,13092.0,03076K10,ABCB,AMERIS BANCORP,30222,522110,3.0,12.0,4.0,2016.0,2016-12-31,2017-01-20


In [78]:
def get_aggregate_earnings_evals(df, pct_thresh=0.06, nom_thresh=6, cum_pct_thresh_1=0.1, 
                      cum_nom_thresh_1=10, cum_pct_thresh_2=0.1, cum_nom_thresh_2=10 ):
    evals = []
    counter = 0
    keys=['move_pct', 'move_nom', 'move_cum_pct_1', 'move_cum_nom_1', 'move_cum_pct_2', 'move_cum_nom_2']
    for row in df.itertuples():
        #print(counter)
        temp = evaluate_earnings(row.rdq, row.permno, row.cusip, pct_thresh=pct_thresh, nom_thresh=nom_thresh, 
                                 cum_pct_thresh_1=cum_pct_thresh_1, cum_pct_thresh_2=cum_pct_thresh_2, 
                                 cum_nom_thresh_2=cum_nom_thresh_2)
        if 'Error' in temp:
            evals.append({'rdq':row.rdq, 'permno':row.permno, 'cusip':row.cusip, 'Error':True})
        else:
            temp = [1 if x == True else 0 for x in temp]
            temp = {key:val for key, val in zip(keys, temp)}
            temp = {**temp, **{'rdq':row.rdq, 'permno':row.permno, 'cusip':row.cusip, 'Error':False}}
            evals.append(temp)
        counter+=1
        if counter%100 ==0:
            print(counter)
    evals = pd.DataFrame(evals)    
    return evals

In [72]:
sample.iloc[84]

permno                         18768
permco                         56785
cusip                       M4R82T10
ticker                          FVRR
comnam      FIVERR INTERNATIONAL LTD
gvkey                          35222
naics                         519130
exchcd                             1
fyr                               12
fqtr                               4
fyearq                          2018
datadate                  2018-12-31
rdq                       2019-06-03
Name: 31904, dtype: object

In [76]:
evaluate_earnings('2019-06-03', 18768, 'M4R82T10')

['Error']

In [81]:
eval_sample = get_aggregate_earnings_evals(sample)

100
200
300
400
500
600


In [82]:
eval_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   move_pct        574 non-null    float64
 1   move_nom        574 non-null    float64
 2   move_cum_pct_1  574 non-null    float64
 3   move_cum_nom_1  574 non-null    float64
 4   move_cum_pct_2  574 non-null    float64
 5   move_cum_nom_2  574 non-null    float64
 6   rdq             600 non-null    object 
 7   permno          600 non-null    float64
 8   cusip           600 non-null    object 
 9   Error           600 non-null    bool   
dtypes: bool(1), float64(7), object(2)
memory usage: 42.9+ KB


In [66]:
4/80

0.05

In [62]:
eval_sample.loc[~eval_sample['move_nom'].isnull()]

,move_pct,move_nom,move_cum_pct_1,move_cum_nom_1,move_cum_pct_2,move_cum_nom_2,rdq,permno,cusip,Error
0,0.0,0.0,0.0,0.0,0.0,0.0,2019-10-31,18369.0,76209310,False
1,1.0,1.0,1.0,0.0,1.0,0.0,2019-08-08,85067.0,57793310,False
4,1.0,0.0,0.0,0.0,0.0,0.0,2017-11-02,13962.0,63009R10,False
5,0.0,0.0,0.0,0.0,0.0,0.0,2016-10-25,35263.0,89840210,False
6,1.0,0.0,0.0,0.0,1.0,0.0,2015-10-27,90199.0,12504L10,False
...,...,...,...,...,...,...,...,...,...,...
79,1.0,0.0,0.0,0.0,0.0,0.0,2017-03-02,80375.0,68162K10,False
80,1.0,0.0,0.0,0.0,0.0,0.0,2019-04-26,14619.0,17878Y10,False
81,1.0,0.0,0.0,0.0,0.0,0.0,2016-02-02,84314.0,G4481U10,False
82,0.0,0.0,0.0,0.0,0.0,0.0,2019-02-26,91727.0,89421B10,False


##### test for big moves around those dates for the 10 trading days before/after the announcement for the last 5 years

In [ ]:
def test_for_moves

In [53]:
'Error' in [1,2,4,'Error', 7]

True

##### Repeat for all companies and compile a table with yes/no labels for each period. 

The final output will be ticker/cusip/permno/company_name and then date and label